# Import the library

In [1]:
import pandas as pd
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
#from textblob import TextBlob

# load the data from google drive

In [17]:
df = pd.read_csv('Data/50k_reviews.csv')

In [18]:
df.head()

score                                               text
0    3.0  good and helpfull read this book is very good ...
1    1.0  Sadly overpriced and irrelevant In spite of it...
2    2.0  Endless rant Howard should have borrowed from ...
3    1.0  Not Quite Hip It's really a shame about the ti...
4    5.0  Journey to the Centre of the Earth Hey! This i...

# Data Preprocessing

In the above-given problem statement we have performed various pre-processing steps on the dataset that mainly dealt with removing stopwords, removing emojis. The text document is then converted into the lowercase for better generalization.

Subsequently, the punctuations were cleaned and removed thereby reducing the unnecessary noise from the dataset. After that, we have also removed the repeating characters from the words.

At last, we then performed Stemming(reducing the words to their derived stems) and Lemmatization(reducing the derived words to their root form known as lemma) with limitation of part of speech for better results.

## data cleaning

In [4]:
df.fillna('', inplace=True)

In [5]:
df.isna().sum()

score    0
text     0
dtype: int64

In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /Users/xin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Making statement text in lower case

In [7]:
df['text']=df['text'].str.lower()
df['text'].head()

0    good and helpfull read this book is very good ...
1    sadly overpriced and irrelevant in spite of it...
2    endless rant howard should have borrowed from ...
3    not quite hip it's really a shame about the ti...
4    journey to the centre of the earth hey! this i...
Name: text, dtype: object

### Restore common acronyms

In [8]:
def restore_common_abbr(caption):
    pat_is = re.compile("(it|he|she|that|this|there|here)(\'s)", re.I)
    pat_s = re.compile("(?<=[a-zA-Z])\'s")  
    pat_s2 = re.compile("(?<=s)\'s?")
    pat_not = re.compile("(?<=[a-zA-Z])n\'t")  # not abbreviation
    pat_would = re.compile("(?<=[a-zA-Z])\'d")  # would abbreviation
    pat_will = re.compile("(?<=[a-zA-Z])\'ll")  # will abbreviation
    pat_am = re.compile("(?<=[I|i])\'m")  # am abbreviation
    pat_are = re.compile("(?<=[a-zA-Z])\'re")  # are abbreviation
    pat_ve = re.compile("(?<=[a-zA-Z])\'ve")  # have abbreviation

    new_text = caption
    new_text = pat_is.sub(r"\1 is", new_text)
    new_text = pat_s.sub("", new_text)
    new_text = pat_s2.sub("", new_text)
    new_text = pat_not.sub(" not", new_text)
    new_text = pat_would.sub(" would", new_text)
    new_text = pat_will.sub(" will", new_text)
    new_text = pat_am.sub(" am", new_text)
    new_text = pat_are.sub(" are", new_text)
    new_text = pat_ve.sub(" have", new_text)
    new_text = new_text.replace('\'', ' ')
    return new_text

In [9]:
df['text']= df['text'].apply(lambda x: restore_common_abbr(x))
df['text'].head()

0    good and helpfull read this book is very good ...
1    sadly overpriced and irrelevant in spite of it...
2    endless rant howard should have borrowed from ...
3    not quite hip it is really a shame about the t...
4    journey to the centre of the earth hey! this i...
Name: text, dtype: object

In [10]:
#not sure how to deal with 'not'
df = pd.DataFrame({'comment':['aaa bbb ccc not verb ddd']}) 

df.comment.str.replace(r'\b(not\s)', 'not_')


/var/folders/0s/xfpwvv6x0fg3pkqm99zxr6z40000gq/T/ipykernel_68815/2573328388.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df.comment.str.replace(r'\b(not\s)', 'not_')


0    aaa bbb ccc not_verb ddd
Name: comment, dtype: object

### Cleaning and removing the above stop words list from the text

In [11]:
STOPWORDS = set(stopwords.words('english'))
print(STOPWORDS)

{'hadn', 'itself', 'whom', 'each', 'haven', 'all', 'needn', 'an', 'didn', 'your', 'same', "weren't", 'now', 'just', "shan't", 'on', "isn't", 'some', 'she', 'down', 'here', 'shouldn', 'into', 'more', "it's", 'but', 'll', 'theirs', 'these', 'for', 'doing', 'which', 'won', 'doesn', 'will', 'yourself', 'him', 'where', "hasn't", 'against', 'hers', 'weren', 'no', 'after', 'over', 'between', 'when', 'while', 'of', 'having', 'being', 'he', 'i', "wouldn't", 'as', 't', 'm', 'in', 'should', "you'll", 'you', 'hasn', 'out', 'any', 'couldn', "you've", 'other', 'has', 'below', 'a', 'yours', 'isn', 'themselves', 'were', 'we', 'and', 'y', "that'll", "didn't", 'himself', 'what', 'at', "hadn't", 'above', 'there', 'its', "needn't", 'own', 'wasn', 'by', "you'd", "aren't", 'his', 'only', 'them', "shouldn't", 'can', 'ourselves', "haven't", 'ma', 'am', 're', 'from', 'our', "should've", 'wouldn', 'did', 'those', 'd', 'under', 'about', 'or', 'off', 'not', 'both', 'most', 'than', 's', 'don', "couldn't", 'aren', 

In [36]:
#create our new stopoing words to keep some words like 'not'  
my_stopwords ={'have', 'my', 'so', 're', 'any', 'only', 'haven', 'between', 'be', 'am', "hadn't", 'its', 've', 'needn', 'you', 'doing', "don't", 'she', 'all', 'against', 'me', 'were', 'y', 'theirs', 'hadn', 'myself', 'should', "didn't", 'our', 'themselves', "should've", "you'd", 'where', 'a', 'hers', 'here', 'are', 'didn', 'below', 'is', 'this', 'ma', "that'll", "mustn't", 'weren', 'wouldn', 'their', 'ours', 'don', 'other', 'same', "won't", 'more', 'how', "isn't", 'few', 'than', 'too', 'in', 'if', 'yours', 'ain', 'yourself', "wouldn't", 'those', "she's", 'until', 'i', 'itself', 'has', 'shouldn', 'your', 'own', 'under', 'further', 'again', 'm', 'but', 'won', 'over', 'no', 'mustn', 'that', 'while', 'during', 'to', 'above', 'such', 'will', 'shan', "wasn't", 'him', 'them', 'was', 'he', 'yourselves', 'there', 'herself', 'for', "doesn't", "couldn't", 'doesn', 'did', 'hasn', 'with', 'some', 'into', 'does', 'nor', 'o', "needn't", 'and', 'most', 'his', 'having', 'after', "you've", 'the', 'they', 'then', 'isn', "shan't", 'once', 't', 'it', "shouldn't", 'been', 'can', 'being', 'of', 'about', 'just', 'down', "hasn't", 'now', 'ourselves', 'd', 'up', 'her', 'had', 'aren', 'what', 'from', "you'll", 'each', 'we', 'whom', "mightn't", 'do', 'because', 'through', 's', "weren't", 'himself', 'who', 'both', 'an', 'by', 'when', 'at', 'as', 'off', 'very', 'these', 'or', "it's", 'mightn', 'before', "you're", "haven't", 'on', 'why', 'll', 'which', 'wasn', "aren't", 'couldn', 'out','``'}

In [37]:
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in my_stopwords])
df['text'] = df['text'].apply(lambda text: cleaning_stopwords(text))
df['text'].head()

0    ['good', 'helpfull', 'read', 'book', 'good', '...
1    ['Sadly', 'overprice', 'irrelevant', 'spite', ...
2    ['Endless', 'rant', 'Howard', 'borrow', 'Denni...
3    ['Not', 'Quite', 'Hip', 'It', "'s", 'really', ...
4    ['Journey', 'Centre', 'Earth', 'Hey', 'This', ...
Name: text, dtype: object

### Cleaning and removing punctuations

In [20]:
import string

def cleaning_punctuations(text):
    new_text=re.sub(r"[%s]+" %string.punctuation, " ",text)
    pat_space=re.compile('\s{2,}')
    new_text=pat_space.sub(" ",text)
    return new_text
df['text']= df['text'].apply(lambda x: cleaning_punctuations(x))
df['text'].head()

0    good helpfull read book good type thats findin...
1    Sadly overpriced irrelevant In spite claim "il...
2    Endless rant Howard borrowed Dennis Miller sta...
3    Not Quite Hip It's really shame time reserch w...
4    Journey Centre Earth Hey! This great book! I a...
Name: text, dtype: object

### Cleaning and removing repeating characters

In [21]:
def cleaning_repeating_char(text):
    return re.sub(r'(.)1+', r'1', text)
df['text'] = df['text'].apply(lambda x: cleaning_repeating_char(x))
df['text'].head()

0    good helpfull read book good type thats findin...
1    Sadly overpriced irrelevant In spite claim "il...
2    Endless rant Howard borrowed Dennis Miller sta...
3    Not Quite Hip It's really shame time reserch w...
4    Journey Centre Earth Hey! This great book! I a...
Name: text, dtype: object

### Cleaning and removing Numeric numbers

In [22]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', ' ', data)
df['text'] = df['text'].apply(lambda x: cleaning_numbers(x))
df['text'].head()

0    good helpfull read book good type thats findin...
1    Sadly overpriced irrelevant In spite claim "il...
2    Endless rant Howard borrowed Dennis Miller sta...
3    Not Quite Hip It's really shame time reserch w...
4    Journey Centre Earth Hey! This great book! I a...
Name: text, dtype: object

### Remove short words

We remove those words which are of little or no use. So, we will select the length of words which we want to remove

In [23]:
def transform_text(text):
    return ' '.join([word for word in text.split() if len(word) > 2])
df['text'] = df['text'].apply(lambda x: transform_text(x))
df['text'].head() 

0    good helpfull read book good type thats findin...
1    Sadly overpriced irrelevant spite claim "illus...
2    Endless rant Howard borrowed Dennis Miller sta...
3    Not Quite Hip It's really shame time reserch w...
4    Journey Centre Earth Hey! This great book! abs...
Name: text, dtype: object

## Tokenization

In [24]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/xin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
from nltk import word_tokenize
df['text']=df['text'].apply(lambda x: word_tokenize(x))

In [26]:
df['text'].head()

0    [good, helpfull, read, book, good, type, thats...
1    [Sadly, overpriced, irrelevant, spite, claim, ...
2    [Endless, rant, Howard, borrowed, Dennis, Mill...
3    [Not, Quite, Hip, It, 's, really, shame, time,...
4    [Journey, Centre, Earth, Hey, !, This, great, ...
Name: text, dtype: object

## stemming

In [14]:
import nltk
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
df['content']= df['content'].apply(lambda x: stemming_on_text(x))
df['content'].head()

KeyError: 'content'

### Applying Lemmatizer

In [27]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/xin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /Users/xin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [29]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/xin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [30]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# get the part of speech
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def tag_lem(doc,lemmatizer):
    if doc[-1] != '.':
        doc.append('.')
    tagged_sent = pos_tag(doc)  # get the part of speech

    new_s = []
    for c in tagged_sent:
        if c[0].isdigit():
            new_s.append("#number")
        elif c[0] not in string.punctuation:
            wordnet_pos = get_wordnet_pos(c[1]) or wordnet.NOUN
            new_s.append(lemmatizer.lemmatize(c[0], pos=wordnet_pos))  
    return new_s

lemmatizer = WordNetLemmatizer()

df['text'] = df['text'].apply(lambda x: tag_lem(x,lemmatizer))
df['text'].head()

0    [good, helpfull, read, book, good, type, thats...
1    [Sadly, overprice, irrelevant, spite, claim, `...
2    [Endless, rant, Howard, borrow, Dennis, Miller...
3    [Not, Quite, Hip, It, 's, really, shame, time,...
4    [Journey, Centre, Earth, Hey, This, great, boo...
Name: text, dtype: object

In [31]:
tag_lem(['borrowed','cats','facing'],lemmatizer)

['borrow', 'cat', 'face']

In [38]:
df.to_pickle("Data/data_cleaned_1")

In [39]:
df

score                                               text
0        3.0  ['good', 'helpfull', 'read', 'book', 'good', '...
1        1.0  ['Sadly', 'overprice', 'irrelevant', 'spite', ...
2        2.0  ['Endless', 'rant', 'Howard', 'borrow', 'Denni...
3        1.0  ['Not', 'Quite', 'Hip', 'It', "'s", 'really', ...
4        5.0  ['Journey', 'Centre', 'Earth', 'Hey', 'This', ...
...      ...                                                ...
49995    5.0  ['Stars', 'Short', 'Easy', 'explanation', 'Fol...
49996    5.0  ['Michael', 'Hague', 'illustration', 'best', '...
49997    5.0  ['THE', 'quot', 'AMERICAN', 'BEAUTY', 'quot', ...
49998    4.0  ['funny', 'quirky', 'This', 'really', 'funny',...
49999    5.0  ['Soldiers', 'Bible', 'old', 'son', 'Army', 'b...

[50000 rows x 2 columns]

## visualization

In [47]:
df['combine']=df['text'].apply(lambda x: ' '.join(x))
all_words = " ".join(sent for sent in df['combine'])

from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=42, max_font_size=100).generate(all_words)

plt.figure(figsize=(15,8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [37]:
words = all_words.split()
counts = {}  
for word in words:
    counts[word] = counts.get(word, 0) + 1 
items = list(counts.items())
items.sort(key=lambda x: x[1], reverse=True)
for i in range(30):
    word, count = items[i] 
    print ("{0:<10}{1:>5}".format(word, count))



book      113091
not       72980
read      50968
one       32479
story     25187
like      21686
would     20633
time      20164
great     20014
make      19779
good      18890
write     18052
get       18040
find      16466
character 16459
love      16357
life      15273
well      15208
think     14710
novel     13835
first     13097
many      13004
much      12239
know      12226
work      12009
people    11784
take      11658
also      11610
way       11574
give      11409
